# Python 2 HSUTCC: Session 7: Special Task

## Special Task (Bonus 8% - Deadline Thursday 27 Nov 2025)

Symbolic differentiation allows to calculate a value of a derivative for an expression in a form of a sequence of symbols. Humans are likely to make mistakes during a long monotonous work and calculating of a direvative even not that hard but tidious process. Systems of symbolic calculations, like in this <a href='https://www.wolframalpha.com/input?i=derivate+x%5E3+++(1+-+x)+/+(1+-+x%5E2)'>example</a> are dedicated to save humanity from the need to spend their time for a potentially wrong calcuations.

We will try to implement a simple system of symbolic differentiation on Python. The main type in our system will be a class `Expr` – an expression. All successor of the class should implement:

- `__call__` method which calculates a value of an expression in a specified context: the context bounds names of variables in the expression with specific values.
- `d` (Leibniz's notation of derivative) method which takes a name of a variable wrt (from “with respect to”) and returns an expression for a derivative with respect to this variable.

In [2]:
from abc import ABC, abstractmethod

class Expr(ABC):
    @abstractmethod
    def __call__(self, **context):
        pass

    @abstractmethod
    def d(self, wrt):
        pass

    @abstractmethod
    def __str__(self):
        pass

**Important**: For each class, implement a `__str__` method which should return a corresponding formula in a form of S-expression.

1. Implement class for two types of expressions: `Const` – a constant and `Var` – a variable. For convinience next we will use not a constructors of classes but their one-letter synonyms:

```python
V = Var
C = Const
```

In [16]:
# Your code here

class Const(Expr):
    def __init__(self, constant):
        self.constant = constant

    def __call__(self, **context):
        return self.constant

    def d(self, wrt):
        return C(0)

    def __str__(self):
        return str(self.constant)


class Var(Expr):
    def __init__(self, variable):
        self.variable = variable    # variable name (string)

    def __call__(self, **context):
        return context[self.variable]    # look up the value

    def d(self, wrt):
        if isinstance(wrt, Var) and self.variable == wrt.variable:
            return C(1)
        return C(0)

    def __str__(self):
        return self.variable


V = Var
C = Const

Example of class usage:

---

```python
C(42)() # print(C(42)) -> 42
```

In [ ]:
print(C(42))

42


---

```python
C(42).d(V("x"))() # print(C(42).d(V("x"))) -> 0
```

In [12]:
print(C(42).d(V("x")))

0


---

```python
V("x")(x=42) # print(V("x")) -> x
```

In [13]:
V("x")(x=42)
print(V("x"))

x


---

```python
V("x").d(V("x"))() # print(V("x").d(V("x"))) -> 1
```

In [17]:
print(V("x").d(V("x")))

1


---

```python
V("x").d(V("y"))() # print(V("x").d(V("x"))) -> 0
```

In [19]:
print(V("x").d(V("y")))

0


---

2. Implement classes for binary operations: `Sum`, `Product`, and `Fraction`. Binary operations by definition work with exactly two operands that’s why a constructor for all binary classes will be the same. It’s convinient to take it out into a separate base class:


In [20]:
class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2

    @abstractmethod
    def __call__(self, **context) -> float:
        pass

    @abstractmethod
    def d(self, wrt: Expr) -> Expr:
        pass

    @abstractmethod
    def __str__(self) -> str:
        pass

class Sum(BinOp):
    def __call__(self, **context) -> float:
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt: Expr) -> Expr:
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self) -> str:
        return f"(+ {self.expr1} {self.expr2})"
    
class Product(BinOp):
    def __call__(self, **context) -> float:
        return self.expr1(**context) * self.expr2(**context)

    def d(self, wrt: Expr) -> Expr:
        # Product rule: (f * g)' = f' * g + f * g'
        return Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(self.expr1, self.expr2.d(wrt))
        )

    def __str__(self) -> str:
        return f"(* {self.expr1} {self.expr2})"
    
class Fraction(BinOp):
    def __call__(self, **context) -> float:
        return self.expr1(**context) / self.expr2(**context)

    def d(self, wrt: Expr) -> Expr:
        # Quotient rule: (f / g)' = (f' * g - f * g') / g^2
        numerator = Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(C(-1), Product(self.expr1, self.expr2.d(wrt)))
        )
        denominator = Product(self.expr2, self.expr2)
        return Fraction(numerator, denominator)

    def __str__(self) -> str:
        return f"(/ {self.expr1} {self.expr2})"

An example of using of some binary operations:

---

```python
# x + x^2
x = V("x")
Sum(x, Product(x, x)).d(x)(x=42) # (+ 1 (+ (* 1 x) (* x 1)))
```

```terminal
85
```

In [24]:
x = V("x")
print(Sum(x, Product(x, x)).d(x))
Sum(x, Product(x, x)).d(x)(x=42)

(+ 1 (+ (* 1 x) (* x 1)))


85

---

```python
Product(x, Sum(x, C(2)))(x=42) # (* x (+ x 2))
```

```terminal
1848
```

In [25]:
Product(x, Sum(x, C(2)))(x=42)

1848

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(x)(x=42, y=24)
```

```terminal
0.14285714285714285
```

In [26]:
Fraction(Product(x, V("y")), Sum(C(42), x)).d(x)(x=42, y=24)

0.14285714285714285

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(V("y"))(x=42, y=24)
```

```terminal
0.5
```

In [27]:
Fraction(Product(x, V("y")), Sum(C(42), x)).d(V("y"))(x=42, y=24)

0.5

---

3. Implement a class `Power` for an operation of an exponentiation operation. For simplicity you can expect that a power – is an expression consisting only of constants.

In [ ]:
class Power(Expr):
    def __init__(self, base: Expr, exponent: Expr) -> None:
        self.base: Expr = base
        self.exponent: Expr = exponent

    def __call__(self, **context) -> float:
        return self.base(**context) ** self.exponent(**context)

    def d(self, wrt: Expr) -> Expr:
        # Only handle exponent as constant for now
        if isinstance(self.exponent, Const):
            n = self.exponent.constant
            # derivative: n * base^(n-1) * base'
            return Product(
                Product(C(n), Power(self.base, C(n - 1))),
                self.base.d(wrt)
            )

    def __str__(self) -> str:
        return f"(** {self.base} {self.exponent})"


---

```python
Power(Fraction(V("x"), C(4)), C(2))(x=42) # (** (/ x 4) 2)
```

```terminal
110.25
```

In [30]:
Power(Fraction(V("x"), C(4)), C(2))(x=42)

110.25

---

```python
Power(Fraction(V("x"), C(4)), C(2)).d(V("x"))(x=42)
```

```terminal
5.25
```

In [36]:
Power(Fraction(V("x"), C(4)), C(2)).d(V("x"))(x=42)

5.25

---

4. Implement an overloading of arithmetic operators for a base class Expr. Overall there are needed to be implement 7 operators: 2 unary and 5 binary:

```terminal
-e          e.__neg__()
+e          e.__pos__()
e1 + e2     e1.__add__(e2)
e1 - e2     e1.__sub__(e2)
e1 * e2     e1.__mul__(e2)
e1 / e2     e1.__truediv__(e2)
e1 ** e2    e1.__pow__(e2)
```

---

In [7]:
class Expr(ABC):
    @abstractmethod
    def __call__(self, **context):
        pass

    @abstractmethod
    def d(self, wrt):
        pass

    @abstractmethod
    def __str__(self):
        pass

    # unary
    def __neg__(self):
        return Product(C(-1), self)

    def __pos__(self):
        return self

    # binary
    def __add__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Sum(self, other)

    def __radd__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Sum(other, self)

    def __sub__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Sum(self, -other)

    def __rsub__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Sum(other, -self)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Product(self, other)

    def __rmul__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Product(other, self)

    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Fraction(self, other)

    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Fraction(other, self)

    def __pow__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Power(self, other)

    def __rpow__(self, other):
        if isinstance(other, (int, float)):
            other = C(other)
        return Power(other, self)


In [8]:
# To test your implementation, put all other classes here
# Restart the kernel, run the above and current cell, then do some testing
# Your code here

class Const(Expr):
    def __init__(self, constant):
        self.constant = constant

    def __call__(self, **context):
        return self.constant

    def d(self, wrt):
        return C(0)

    def __str__(self):
        return str(self.constant)


class Var(Expr):
    def __init__(self, variable):
        self.variable = variable

    def __call__(self, **context):
        return context[self.variable]

    def d(self, wrt):
        if isinstance(wrt, Var) and self.variable == wrt.variable:
            return C(1)
        return C(0)

    def __str__(self):
        return self.variable


V = Var
C = Const

class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2

    @abstractmethod
    def __call__(self, **context) -> float:
        pass

    @abstractmethod
    def d(self, wrt: Expr) -> Expr:
        pass

    @abstractmethod
    def __str__(self) -> str:
        pass

class Sum(BinOp):
    def __call__(self, **context) -> float:
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt: Expr) -> Expr:
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self) -> str:
        return f"(+ {self.expr1} {self.expr2})"
    
class Product(BinOp):
    def __call__(self, **context) -> float:
        return self.expr1(**context) * self.expr2(**context)

    def d(self, wrt: Expr) -> Expr:
        # Product rule: (f * g)' = f' * g + f * g'
        return Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(self.expr1, self.expr2.d(wrt))
        )

    def __str__(self) -> str:
        return f"(* {self.expr1} {self.expr2})"
    
class Fraction(BinOp):
    def __call__(self, **context) -> float:
        return self.expr1(**context) / self.expr2(**context)

    def d(self, wrt: Expr) -> Expr:
        # Quotient rule: (f / g)' = (f' * g - f * g') / g^2
        numerator = Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(C(-1), Product(self.expr1, self.expr2.d(wrt)))
        )
        denominator = Product(self.expr2, self.expr2)
        return Fraction(numerator, denominator)

    def __str__(self) -> str:
        return f"(/ {self.expr1} {self.expr2})"
    
class Power(Expr):
    def __init__(self, base: Expr, exponent: Expr) -> None:
        self.base: Expr = base
        self.exponent: Expr = exponent

    def __call__(self, **context) -> float:
        return self.base(**context) ** self.exponent(**context)

    def d(self, wrt: Expr) -> Expr:
        # Only handle exponent as constant for now
        if isinstance(self.exponent, Const):
            n = self.exponent.constant
            # derivative: n * base^(n-1) * base'
            return Product(
                Product(C(n), Power(self.base, C(n - 1))),
                self.base.d(wrt)
            )

    def __str__(self) -> str:
        return f"(** {self.base} {self.exponent})"


```python
((C(1) - V("x")) ** C(3) + V("x"))(x=12)
```

```terminal
-1319
```

---

In [9]:
x = V("x")
expr = (C(1) - x) ** C(3) + x
print(expr)          # (+ (** (- 1 x) 3) x)
print(expr(x=12))    # evaluates numerically


(+ (** (+ 1 (* -1 x)) 3) x)
-1319
